In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.metrics import (
                mean_absolute_error, 
                mean_squared_error,
                median_absolute_error,
                max_error,
                r2_score,
                completeness_score,
                fowlkes_mallows_score,
                homogeneity_score,
                v_measure_score )
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


# Inicial verificação dos dados 
__________
Como não há valores vazios, podemos não nos preocupar com valores nulos e tratamentos desses dados.

Para aprendizado supervisionado foi escolhido o atributo/coluna 'preço' (price) para tentar prever os seus dados através de aprendizado supervisionado.

Logo dado o CEP, tipo de propriedade e quantidade de quarto tentar prever qual será seu preço.
_________
Posteriormente, será feito o contrário será colocado a coluna preço e tentará descobrir se é casa ou é "unit".  
____________
CEP será desconsiderado depois, para mais testes, uma vez que CEP não está por bairro e sim algo mais detalhado, tornando mais dificil seu uso.
E feito o teste para descobrir o preço e o tipo de imóvel.




In [ ]:
df=pd.read_csv('../input/property-sales/raw_sales.csv')

print(df.head())
print(df.isna().sum())
print(df.info())


# **Análise atemporal**
______________
Buscou-se usar somentes os dados mais recentes, desde de 2019, para que o não ficasse muito grande. E assim, perde-se a temporalidade dos dados, na verdade usa-se os dados de forma atemporal.

In [ ]:

df_recente = df[(df['datesold'] > '2017-01-01 00:00:00')]
# drop da mesma
df_recente=df_recente.drop('datesold',axis=1)

print(df_recente.head())
print(df_recente.info())

print(df_recente.price.count())

In [ ]:
ax = sns.countplot(x="bedrooms", data=df_recente)
for p in ax.patches:
    ax.annotate('{}'.format(p.get_height()), (p.get_x()+0.15, p.get_height()+1))

In [ ]:
_ = sns.swarmplot(x='propertyType', y='price', data=df_recente)


# Label the axes
_ = plt.xlabel('Tipo de propriedade')
_ = plt.ylabel('Preço')


# Show the plot
plt.show()

In [ ]:
_ = sns.swarmplot(x='bedrooms', y='price', data=df_recente)


# Label the axes
_ = plt.xlabel('Quartos')
_ = plt.ylabel('Preço')


# Show the plot
plt.show()

# Aprendizado supervisionado

Nessa parte, irá salvar a coluna de interesse e usa-lá para treinos e ver qual modelo melhor se adapta.
Separar em treino e teste.
Por costume teste terá 20% do dataset.


In [ ]:
df_recente['propertyType'].replace('unit', 0,inplace=True)
df_recente['propertyType'].replace('house', 1,inplace=True)

In [ ]:
# Selecao da coluna a ser treinada
preco = df_recente.price
# drop da mesma
df1=df_recente.drop('price',axis=1)
print(df1.info())
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(df1, preco, test_size = 0.2)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Métodos de treino

Serão usados métodos de regressão. Buscando de sklearn alguns modelos usados como regressão linear, logit, lasso, Ridge (disponíveis na biblioteca sklearn).

### Métricas usadas

Serão as que são usadas comumente para métodos de regressão:

* Max Error 

Representa o valor máximo de erro entre o predito e o gabarito (true_value). Ou seja, max (|Y_predito - Gabarito|)

* Mean Absolute Error 

Erro médio absoluto, o nome já é explicativo: Soma-se o total de erros e divide-se pelo número de amostras. logo :
 $ \frac{\sum (Y_{predito} - Y_{gabarito})}{n_{amostras}} $

* Mean Squared Error 

Erro médio quadrático tem como base o erro médio absoluto, mas ao invés de pegar o valor absoluto, usa-se o valor quadrático desta diferença. logo :
 $ \frac{\sum \sqrt{(Y_{predito} - Y_{gabarito})^{2}}}{n_{amostras}} $

* Median Absolute Error

Nesse caso, pega-se os erros e se ordena e encontrar a tendência central do mesmo. Ou seja, as amostras em $X[\frac{n}{2}]$ ou $X\frac{[\frac{n}{2}] + X[\frac{n}{2} + 1]}{2}$

* Mean Absolute Percentage Error

MAPE é a soma dos erros absolutos individuais divididos pela demanda (cada período separadamente).
$  MAPE = \frac{1}{n} \sum \frac{Y_{Predito} - Y_{gabarito}}{Y_{gabarito}} $

In [ ]:
modeloRegrLinear =  LinearRegression().fit(train_X, train_y)
y_pred =  modeloRegrLinear.predict(val_X)
modeloRidge = Ridge().fit(train_X, train_y)
y_predRidge =  modeloRidge.predict(val_X)
modeloLasso = Lasso().fit(train_X, train_y)
y_predLasso =  modeloLasso.predict(val_X)

In [ ]:
print("Regressão linear \n\t\t\t\t Acurácia treino: ", modeloRegrLinear.score(train_X,train_y),
     "\n\t\t\t\t Acurácia validação: ", modeloRegrLinear.score(val_X,val_y),
     "\n\t\t\t\t Erro médio absoluto: ", mean_absolute_error(val_y,y_pred),
      "\n\t\t\t\t Erro Percentual médio absoluto: ", mean_absolute_percentage_error(val_y,y_pred),
     "\n\t\t\t\t Erro médio quadrático: ", mean_squared_error(val_y,y_pred),
      "\n\t\t\t\t Erro mediano absoluto: ", median_absolute_error(val_y,y_pred),
      "\n\t\t\t\t Máximo erro: ", max_error(val_y,y_pred),
      "\n\t\t\t\t R2 score: ", r2_score(val_y,y_pred)
         )
print("Regressão Ridge \n\t\t\t\t Acurácia treino: ", modeloRidge.score(train_X,train_y),
     "\n\t\t\t\t Acurácia validação: ", modeloRidge.score(val_X,val_y),
     "\n\t\t\t\t Erro médio absoluto: ", mean_absolute_error(val_y,y_predRidge),
     "\n\t\t\t\t Erro Percentual médio absoluto: ", mean_absolute_percentage_error(val_y,y_predRidge),
     "\n\t\t\t\t Erro médio quadrático: ", mean_squared_error(val_y,y_predRidge),
     "\n\t\t\t\t Erro mediano absoluto: ", median_absolute_error(val_y,y_predRidge),
     "\n\t\t\t\t Máximo erro: ", max_error(val_y,y_predRidge),
     "\n\t\t\t\t R2 score: ", r2_score(val_y,y_predRidge))

print("Regressão Lasso \n\t\t\t\t Acurácia treino: ", modeloLasso.score(train_X,train_y),
     "\n\t\t\t\t Acurácia validação: ", modeloLasso.score(val_X,val_y),
     "\n\t\t\t\t Erro médio absoluto: ", mean_absolute_error(val_y,y_predLasso),
     "\n\t\t\t\t Erro Percentual médio absoluto: ", mean_absolute_percentage_error(val_y,y_predLasso),
     "\n\t\t\t\t Erro médio quadrático: ", mean_squared_error(val_y,y_predLasso),
     "\n\t\t\t\t Erro mediano absoluto: ", median_absolute_error(val_y,y_predLasso),
     "\n\t\t\t\t Máximo erro: ", max_error(val_y,y_predLasso),
      "\n\t\t\t\t R2 score: ", r2_score(val_y,y_predLasso))


# Aprendendo o tipo de imóvel

Ao invés de tentar aprender o preço do imóvel tentar descobrir o imóvel se é unidade ou apartamento.

In [ ]:
# Selecao da coluna a ser treinada
tipo = df_recente.propertyType
# drop da mesma
df2=df_recente.drop('propertyType',axis=1)
print(df1.info())
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(df2, tipo, random_state = 1,test_size = 0.2)

In [ ]:
modeloRegrLinear =  LinearRegression().fit(train_X, train_y)
y_pred =  modeloRegrLinear.predict(val_X)

modeloRidge = Ridge().fit(train_X, train_y)
y_predRidge =  modeloRidge.predict(val_X)

modeloLogRegression = LogisticRegression().fit(train_X,train_y)
y_predLog =  modeloLogRegression.predict(val_X)

In [ ]:
print("Regressão linear \n\t\t\t\t Acurácia treino: ", modeloRegrLinear.score(train_X,train_y),
     "\n\t\t\t\t Acurácia validação: ", modeloRegrLinear.score(val_X,val_y),
     "\n\t\t\t\t Erro médio absoluto: ", mean_absolute_error(val_y,y_pred),
     "\n\t\t\t\t Erro Percentual médio absoluto: ", mean_absolute_percentage_error(val_y,y_pred),      
     "\n\t\t\t\t Erro médio quadrático: ", mean_squared_error(val_y,y_pred),
     "\n\t\t\t\t Erro mediano absoluto: ", median_absolute_error(val_y,y_pred),
     "\n\t\t\t\t Máximo erro: ", max_error(val_y,y_pred),
      "\n\t\t\t\t R2 score: ", r2_score(val_y,y_pred)
         )
print("Regressão Ridge \n\t\t\t\t Acurácia treino: ", modeloRidge.score(train_X,train_y),
     "\n\t\t\t\t Acurácia validação: ", modeloRidge.score(val_X,val_y),
     "\n\t\t\t\t Erro médio absoluto: ", mean_absolute_error(val_y,y_predRidge),
     "\n\t\t\t\t Erro Percentual médio absoluto: ", mean_absolute_percentage_error(val_y,y_predRidge),
     "\n\t\t\t\t Erro médio quadrático: ", mean_squared_error(val_y,y_predRidge),
     "\n\t\t\t\t Erro mediano absoluto: ", median_absolute_error(val_y,y_predRidge),
     "\n\t\t\t\t Máximo erro: ", max_error(val_y,y_predRidge),
      "\n\t\t\t\t R2 score: ", r2_score(val_y,y_predRidge))

print("Regressão Regressão Logística \n\t\t\t\t Acurácia treino: ", modeloLogRegression.score(train_X,train_y),
     "\n\t\t\t\t Acurácia validação: ", modeloLogRegression.score(val_X,val_y),
     "\n\t\t\t\t Erro médio absoluto: ", mean_absolute_error(val_y,y_predLog),
     "\n\t\t\t\t Erro Percentual médio absoluto: ", mean_absolute_percentage_error(val_y,y_predLog),
     "\n\t\t\t\t Erro médio quadrático: ", mean_squared_error(val_y,y_predLog),
     "\n\t\t\t\t Erro mediano absoluto: ", median_absolute_error(val_y,y_predLog),
     "\n\t\t\t\t Máximo erro: ", max_error(val_y,y_predLog),
      "\n\t\t\t\t R2 score: ", r2_score(val_y,y_predLog))

# Removendo o CEP (PostCode)

Agora, removeremos o PostCode e faremos os mesmos modelos e métricas. 

In [ ]:
df_recente_semCEP=df_recente.drop('postcode',axis=1)
# Selecao da coluna a ser treinada
preco_semCEP = df_recente_semCEP.price
# drop da mesma
df1_semCEP=df_recente_semCEP.drop('price',axis=1)
print(df1_semCEP.info())
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(df1_semCEP, preco_semCEP, test_size = 0.2)


In [ ]:
modeloRegrLinear =  LinearRegression().fit(train_X, train_y)
y_pred =  modeloRegrLinear.predict(val_X)

modeloRidge = Ridge().fit(train_X, train_y)
y_predRidge =  modeloRidge.predict(val_X)

modeloLasso = Lasso().fit(train_X, train_y)
y_predLasso =  modeloLasso.predict(val_X)


In [ ]:
print("Regressão linear \n\t\t\t\t Acurácia treino: ", modeloRegrLinear.score(train_X,train_y),
     "\n\t\t\t\t Acurácia validação: ", modeloRegrLinear.score(val_X,val_y),
     "\n\t\t\t\t Erro médio absoluto: ", mean_absolute_error(val_y,y_pred),
     "\n\t\t\t\t Erro Percentual médio absoluto: ", mean_absolute_percentage_error(val_y,y_pred),
      "\n\t\t\t\t Erro médio quadrático: ", mean_squared_error(val_y,y_pred),
      "\n\t\t\t\t Erro mediano absoluto: ", median_absolute_error(val_y,y_pred),
      "\n\t\t\t\t Máximo erro: ", max_error(val_y,y_pred),
      "\n\t\t\t\t R2 score: ", r2_score(val_y,y_pred)
         )
print("Regressão Ridge \n\t\t\t\t Acurácia treino: ", modeloRidge.score(train_X,train_y),
     "\n\t\t\t\t Acurácia validação: ", modeloRidge.score(val_X,val_y),
     "\n\t\t\t\t Erro médio absoluto: ", mean_absolute_error(val_y,y_predRidge),      
     "\n\t\t\t\t Erro Percentual médio absoluto: ", mean_absolute_percentage_error(val_y,y_predRidge),
     "\n\t\t\t\t Erro médio quadrático: ", mean_squared_error(val_y,y_predRidge),
     "\n\t\t\t\t Erro mediano absoluto: ", median_absolute_error(val_y,y_predRidge),
     "\n\t\t\t\t Máximo erro: ", max_error(val_y,y_predRidge),
      "\n\t\t\t\t R2 score: ", r2_score(val_y,y_predRidge))

print("Regressão Lasso \n\t\t\t\t Acurácia treino: ", modeloLasso.score(train_X,train_y),
     "\n\t\t\t\t Acurácia validação: ", modeloLasso.score(val_X,val_y),
     "\n\t\t\t\t Erro médio absoluto: ", mean_absolute_error(val_y,y_predLasso),
     "\n\t\t\t\t Erro Percentual médio absoluto: ", mean_absolute_percentage_error(val_y,y_predLasso),
     "\n\t\t\t\t Erro médio quadrático: ", mean_squared_error(val_y,y_predLasso),
     "\n\t\t\t\t Erro mediano absoluto: ", median_absolute_error(val_y,y_predLasso),
    "\n\t\t\t\t Máximo erro: ", max_error(val_y,y_predLasso),
      "\n\t\t\t\t R2 score: ", r2_score(val_y,y_predLasso))


In [ ]:
# Selecao da coluna a ser treinada
tipo_semCEP = df_recente_semCEP.propertyType
# drop da mesma
df2_semCEP=df_recente_semCEP.drop('propertyType',axis=1)
print(df2_semCEP.info())
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(df2_semCEP, tipo_semCEP, random_state = 1,test_size = 0.2)

In [ ]:
modeloRegrLinear =  LinearRegression().fit(train_X, train_y)
y_pred =  modeloRegrLinear.predict(val_X)

modeloRidge = Ridge().fit(train_X, train_y)
y_predRidge =  modeloRidge.predict(val_X)

modeloLogRegression = LogisticRegression().fit(train_X,train_y)
y_predLog =  modeloLogRegression.predict(val_X)

In [ ]:
print("Regressão linear \n\t\t\t\t Acurácia treino: ", modeloRegrLinear.score(train_X,train_y),
     "\n\t\t\t\t Acurácia validação: ", modeloRegrLinear.score(val_X,val_y),
     "\n\t\t\t\t Erro médio absoluto: ", mean_absolute_error(val_y,y_pred),
     "\n\t\t\t\t Erro Percentual médio absoluto: ", mean_absolute_percentage_error(val_y,y_pred),
     "\n\t\t\t\t Erro médio quadrático: ", mean_squared_error(val_y,y_pred),
     "\n\t\t\t\t Erro mediano absoluto: ", median_absolute_error(val_y,y_pred),
     "\n\t\t\t\t Máximo erro: ", max_error(val_y,y_pred),
      "\n\t\t\t\t R2 score: ", r2_score(val_y,y_pred)
         )
print("Regressão Ridge \n\t\t\t\t Acurácia treino: ", modeloRidge.score(train_X,train_y),
     "\n\t\t\t\t Acurácia validação: ", modeloRidge.score(val_X,val_y),
     "\n\t\t\t\t Erro médio absoluto: ", mean_absolute_error(val_y,y_predRidge),
     "\n\t\t\t\t Erro Percentual médio absoluto: ", mean_absolute_percentage_error(val_y,y_predRidge),
      "\n\t\t\t\t Erro médio quadrático: ", mean_squared_error(val_y,y_predRidge),
     "\n\t\t\t\t Erro mediano absoluto: ", median_absolute_error(val_y,y_predRidge),
     "\n\t\t\t\t Máximo erro: ", max_error(val_y,y_predRidge),
      "\n\t\t\t\t R2 score: ", r2_score(val_y,y_predRidge))

print("Regressão Regressão Logística \n\t\t\t\t Acurácia treino: ", modeloLogRegression.score(train_X,train_y),
     "\n\t\t\t\t Acurácia validação: ", modeloLogRegression.score(val_X,val_y),
     "\n\t\t\t\t Erro médio absoluto: ", mean_absolute_error(val_y,y_predLog),
     "\n\t\t\t\t Erro Percentual médio absoluto: ", mean_absolute_percentage_error(val_y,y_predLog),
     "\n\t\t\t\t Erro médio quadrático: ", mean_squared_error(val_y,y_predLog),
     "\n\t\t\t\t Erro mediano absoluto: ", median_absolute_error(val_y,y_predLog),
     "\n\t\t\t\t Máximo erro: ", max_error(val_y,y_predLog),
      "\n\t\t\t\t R2 score: ", r2_score(val_y,y_predLog))

__________________________


In [ ]:
zoo2 = pd.read_csv("../input/zoo-animals-extended-dataset/zoo2.csv")
zoo3 = pd.read_csv("../input/zoo-animals-extended-dataset/zoo3.csv")

# Aprendizado não supervisionado
______________

Para esse caso escolheu-se um dataset contendo animais de um zoologico de São Paulo.

Essa tabela contêm o nome de cada bixo e seu ID.  Composto por 16 atributos que dizem se é ou não é.

* animal_name animal id (unique for each instance)
* hairif -- se tem cabelo
* feathersif -- se tem penas
* eggsif -- se poe ovos
* milkif -- se dá leite
* airborne -- se voa
* aquaticif -- se é aquático
* predatorif -- se é predador
* toothedif -- se tem dentes
* backboneif -- se tem ossos
* breathesif -- se "respira ar"
* venomousif -- se é venosos
* finsif -- se tem barbatana
* legsnumber -- numeros de pernas: {0,2,4,5,6,8}
* tailif -- se tem pescoço
* domesticif -- se é doméstico
* catsizeif -- tamanho referente em unidades de gato
* class_typea -- classe pertecente. 


In [ ]:
# peeking at the dataset
print(zoo3.head())
#Descriptive stats of the variables in data
print(zoo3.describe())
# verificando se dados nulos
print(zoo3.isna().sum())

In [ ]:
animal_name = zoo3.animal_name
class_t = zoo3.class_type
zoo3_Km = zoo3.drop(columns=['animal_name','class_type'],axis=1)

In [ ]:
plt.figure(figsize=(10, 8))

wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(zoo3_Km)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 11), wcss)
plt.title('The Elbow Method')
plt.xlabel('Número de Clusters')
plt.ylabel('Coeficiente de Silhueta')
plt.show()

# Número de cluster

Dados as colunas que contêm dados sobre os animais catolagados.

Tentaremos dividir em 6 grupos:
* Mamíferos
* Insetos
* Reptéis 
* anfíbios
* Aves 
* Peixes 

In [ ]:
kmeans = KMeans(n_clusters=6, random_state = 0)
kmeans.fit(zoo3_Km)

In [ ]:
labels = kmeans.predict(zoo3_Km)
centroides = kmeans.cluster_centers_
own_labels = np.array(['Mamíferos', 'Reptéis e Anfíbios', 'Aves', 'Peixe'])
print("Labels: \n", labels);
print("Centroides: \n", centroides);

print(animal_name)

dummy_data3 = {
        'nome': animal_name,
        'labels': labels}
df_label = pd.DataFrame(dummy_data3, columns = ['nome', 'labels'])
group = df_label.groupby('labels')

#df_label.to_csv('labels.csv')
labels_true = [1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,5,5,5,5,5,5,5,5,3,3,
3,3,3,3,3,3,0,0,0,0,0,0,4,4,4,4,4,4,3,4,4]

print(group.count())

for key, item in group:
    print(group.get_group(key), "\n\n")


In [ ]:
tsne_out = TSNE(n_components=2,  n_iter=1000, init='pca').fit_transform(zoo3_Km)
plt.figure(figsize=(13,7))
plt.scatter(tsne_out[:,0], tsne_out[:,1], c=kmeans.labels_,cmap='cividis');

Por fim, obteve-se 6 grupos conforme abaixo:

### Label 0 - Anfíbios

* 40                  jabuti       0 (estranho ao ninho)
* 41            jacare-coroa       0 (estranho ao ninho)
* 45                 tracaja       0 (OK)
* 55  perereca-de-alcatrazes       0 (OK)
* 56          ra-flecha-azul       0 (OK)
* 57              ra-pimenta       0 (OK)
* 58    sapo-barriga-de-fogo       0 (OK)
* 59             sapo-cururu       0 (OK)
* 60          sapo-de-chifre       0 (OK)

### Label 1 - Mamíferos
* 0                anta       1 (OK)
* 1            ariranha       1 (OK)
* 3               bugio       1 (OK)
* 4    cachorro-vinagre       1 (OK)
* 5           chimpanze       1 (OK)
* 6       gato-maracaja       1 (OK)
* 7         jaguatirica       1 (OK)
* 8          lobo-guara       1 (OK)
* 9       macaco-aranha       1 (OK)
* 10   macaco-barrigudo       1 (OK)
* 11  mico-leao-dourado       1 (OK)
* 12     mono-carvoeiro       1 (OK)
* 13       onca-pintada       1 (OK)
* 14        orangotango       1 (OK)
* 15          peixe-boi       1 (OK)
* 16           queixada       1 (OK)
* 17  tamandua-bandeira       1 (OK)
* 18     urso-de-oculos       1  (OK)

### Label 2 - Aves
* 9         aguia-cinzenta       2 (OK)
* 20         aracari-banana       2 (OK)
* 21             arara-azul       2 (OK)
* 22          arara-caninde       2 (OK)
* 23                  chaua       2 (OK)
* 24                    ema       2 (OK)
* 25           gaviao-pombo       2 (OK)
* 26                  guara       2 (OK)
* 27                 harpia       2 (OK)
* 28               jacurutu       2 (OK)
* 29              jacutinga       2 (OK)
* 30        jandaia-amarela       2 (OK)
* 31                 macuco       2 (OK)
* 32             murucututu       2 (OK)
* 33                  mutum       2 (OK)
* 34  papagaio-de-cara-roxa       2 (OK)
* 35         pato-de-crista       2 (OK)
* 36                   pavo       2 (OK)
* 37   tucano-de-bico-preto       2 (OK)
* 38              urubu-rei       2  (OK)

### Label 3 - Peixes
* 47                     baiacu       3 (OK)
* 48      cascudinho-de-caverna       3 (OK)
* 49                    lambari       3 (OK)
* 50                   matrinxa       3 (OK)
* 51                   pirarucu       3 (OK)
* 52                 raia-chita       3 (OK)
* 53                   tambaqui       3 (OK)
* 54             tubarao-raposa       3 (OK)
* 67  caracol-da-mata-atlantica       3 (OK)

### Label 4 - Insetos
* 61         abelha       4 (OK)
* 62       joaninha       4 (OK)
* 63       mariposa       4 (OK)
* 64      pirilampo       4 (OK)
* 65          vespa       4 (OK)
* 66      bicho-pau       4 (OK)
* 68  caranguejeira       4 (OK)
* 69    sauva-limao       4 (OK)

### Label 5 - Reptéis
* 2   boto-cor-de-rosa       5 (Estranho ao ninho)
* 39        cobra-cipo       5 (Ok)
* 42    jararaca-ilhoa       5 (Ok)
* 43            jiboia       5 (Ok)
* 44            sucuri       5 (Ok)
* 46    urutu-cruzeiro       5  (Ok)


# Métricas
__________

* Completude

Refere-se como os membros de um clusters foram preditos para um mesmo cluster (não necessariamente o cluster correto mas agrupados juntos). Calcula-se usando a entropia dos eventos. 

$completude = 1 - \frac{H(Predito|Correto)}{H(Correto)}$

* Homogenidade 

Cada classe (cluster correto) contem membros  de um mesmo cluster predito, pode se imaginar como o inverso de completude. O que não implica em $Completude + Homogenidade = 1 $.  Logo, é a razão de um membro de uma classe dada ser assinalado ao mesmo cluster.
Mas temos que $ Completude(Predito, Gabarito) == Homogenidade(Gabarito, Predito) $

* V-measure

É a média harmônica de homogenidade e completude.

Dado que : Completude = c e Homogenidade = h

$V_{Measure} = (1 + \beta) \frac{h \times c}{\beta \times h + c}$

* Acurácia fowlkes

É definido como a média geométrica entre precisão e recall:

$Fowlkes = \frac{Verdadeiro_{Positivo}}{ \sqrt{((Verdadeiro_{Positivo} + Falso_{Positivo}) * (Verdadeiro_{Positivo} + Falso_{Negativo} ))}}$

In [ ]:
print("Métricas da Cluster \n\t\t\t\t Completude: ", completeness_score(labels_true,labels),
     "\n\t\t\t\t Acurácia fowlkes: ", fowlkes_mallows_score(labels_true,labels),
     "\n\t\t\t\t Homogenidade: ", homogeneity_score(labels_true,labels),
     "\n\t\t\t\t V-measure: ", v_measure_score(labels_true,labels) )

### Aumentando número de cluster para 7



In [ ]:
kmeans2 = KMeans(n_clusters=7, random_state = 0)
kmeans2.fit(zoo3_Km)

In [ ]:
labels2 = kmeans2.predict(zoo3_Km)
centroides2 = kmeans2.cluster_centers_
print("Labels: \n", labels2);
print("Centroides: \n", centroides2);

dummy_data3 = {
        'nome': animal_name,
        'labels': labels2}
df_label = pd.DataFrame(dummy_data3, columns = ['nome', 'labels'])
group2 = df_label.groupby('labels')

print(group2.count())

for key, item in group2:
    print(group2.get_group(key), "\n\n")


In [ ]:
tsne_out = TSNE(n_components=2,  n_iter=1000, init='pca').fit_transform(zoo3_Km)
plt.figure(figsize=(13,7))
plt.scatter(tsne_out[:,0], tsne_out[:,1], c=kmeans2.labels_,cmap='coolwarm');

In [ ]:
print("Métricas da Cluster \n\t\t\t\t Completude: ", completeness_score(class_t,labels2),
     "\n\t\t\t\t Acurácia fowlkes: ", fowlkes_mallows_score(class_t,labels2),
     "\n\t\t\t\t Homogenidade: ", homogeneity_score(class_t,labels2),
     "\n\t\t\t\t V-measure: ", v_measure_score(class_t,labels2) )

-------------------------------

# Aprendizado por reforço - Racing King
------------------------------


O dataset escolhido é um jogo de Xadrez, na verdade uma variante do jogo de xadrez denominada Corrida dos Reis. Contendo um etapa de treino e outra de validação.
Foi escolhido esse dataset pois reforço envolve caminhos bons que te levam a vitória e caminhos ruins que te levam a derrota.



In [ ]:
racing_king_train = pd.read_csv("../input/racingkings/racing_king_train.csv")
racing_king_validate = pd.read_csv("../input/racingkings/racing_king_validate.csv")

In [ ]:
print(racing_king_train.head())
print((racing_king_train.info()))

Nesse dataset há duas colunas

* fen -- fen é o estado atual do jogo e assim por dizer como estão posicionadas as peças suas e do oponente.

* score -- é a pontuação da aquela "snapshot" do jogo X(fen) = score.


Logo, a ideia seria ir reforçando o aprendizado mostrando que tal jogada é de pouco valor e outro é de maior pontuação podendo direcionar as jogadas do bot.
________________

## Métricas de aprendizado por reforço

* Regret -> Uma métrica que diz quantas jogadas "nesse caso do xadrez" seriam economizadas. Ou seja, as vantagens que se levaria se tomasse o caminho ótimo.


* Reability -> confiabilidade, quanto aquela conhecimento realmente levou ao sucesso, ou se for ruim quanto levou ao fracasso.
